In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
from tqdm import tqdm
from astropy.io import fits
import datetime
import time,sys
import astropy.units as u
import numpy as np
import matplotlib.pylab as plt
from IPython.core.display import display, HTML
import sys,os,warnings
sys.path.append('/Users/gstrampelli/PycharmProjects/Giovanni/src/StraKLIP/straklip/utils')
sys.path.append('/Users/gstrampelli/PycharmProjects/Giovanni/pyklip/pyklip')
sys.path.append('/Users/gstrampelli/PycharmProjects/Giovanni/src/StraKLIP/straklip/steps')
sys.path.append('/Users/gstrampelli/PycharmProjects/Giovanni/src/StraKLIP/straklip/')

display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/pm/1ctsvhj16nq8d3h53dh_18jc0000gn/T/ipykernel_15370/669181042.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
unq_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/targets_drc.csv')
unq_target_df

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr
0,J155150.21-213457.4,iexn13010_drc.fits,F814W,237.959171,-21.582686,766.682062,870.519962,22.277353,0.021258,1.416799,227.932175
1,J164636.12-231337.6,iexn01010_drc.fits,F814W,251.650498,-23.227155,769.100793,866.146099,20.840983,0.005901,4.008029,792.416809
2,J164636.12-231337.6,iexn01020_drc.fits,F850LP,251.650498,-23.227156,768.870891,866.488792,19.749264,0.006932,3.617601,600.396423
3,J155416.68-263018.1,iexn02010_drc.fits,F814W,238.569423,-26.505131,762.415167,869.798303,21.448087,0.010977,5.588474,411.439362
4,J155416.68-263018.1,iexn02020_drc.fits,F850LP,238.569423,-26.505132,762.213142,870.180384,20.330281,0.012019,5.789669,338.973694
...,...,...,...,...,...,...,...,...,...,...,...
109,J042630.56+244355.5,iexn57020_drc.fits,F850LP,66.627360,24.732046,772.161171,860.967594,17.388929,0.000777,9.944833,5503.881348
110,J041548.09+291132.9,iexn59010_drc.fits,F814W,63.950387,29.192447,771.356066,869.879264,19.495704,0.001705,3.358237,2707.892578
111,J041548.09+291132.9,iexn59020_drc.fits,F850LP,63.950389,29.192447,771.125543,870.259044,18.268471,0.001755,3.136259,2408.281494
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496


In [3]:
unq_target_df.loc[unq_target_df.target.isin(['J164636.12-231337.6 ','J042705.86+261520.3','J042728.00+261204.7'])]

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr
104,J042705.86+261520.3,iexn54010_drc.fits,F814W,66.774431,26.255547,775.939388,859.286880,21.223795,0.008219,13.334347,413.132355
105,J042705.86+261520.3,iexn54020_drc.fits,F850LP,66.774431,26.255554,775.359033,859.690031,20.284122,0.010910,12.666919,302.929840
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496
113,J042728.00+261204.7,iexn60020_drc.fits,F850LP,66.866686,26.201262,770.160321,868.634002,17.932862,0.001287,2.555961,3282.342041


In [4]:
unq_target_df.loc[unq_target_df.target.isin(['J042705.86+261520.3','J164636.12-231337.6 '])]

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr
104,J042705.86+261520.3,iexn54010_drc.fits,F814W,66.774431,26.255547,775.939388,859.286880,21.223795,0.008219,13.334347,413.132355
105,J042705.86+261520.3,iexn54020_drc.fits,F850LP,66.774431,26.255554,775.359033,859.690031,20.284122,0.010910,12.666919,302.929840


In [5]:
mvs_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/targets_flc.csv')
mvs_target_df

,TARGET,file,x,y,dist,snr
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185
...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008


In [6]:
path2fits='/Users/gstrampelli/PycharmProjects/Giovanni/work/data/FFP/fits/'
for file in tqdm(glob(path2fits+'/*')):
    fitsname=file.split('/')[-1].split('.')[0]
    hdul=fits.open(path2fits+fitsname+'.fits')
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']
    
    dateobs=hdul[0].header['DATE-OBS'].split('-')
    timeobs=hdul[0].header['TIME-OBS'].split(':')
    
    # df=pd.read_csv(file)
    date_time = datetime.datetime(int(dateobs[0]), int(dateobs[1]), int(dateobs[2]), int(timeobs[0]), int(timeobs[1]), int(timeobs[2]))
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','unittime']=time.mktime(date_time.timetuple())
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','fitsroot']=fitsname.split('_')[0]
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','ccd']=hdul[1].header['CCDCHIP']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','visit']=str(fitsname[4:6])
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','exptime']=hdul[0].header['EXPTIME']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','pav3']=hdul[0].header['PA_V3']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','rota']=hdul[1].header['ORIENTAT']
 

100%|████████████████████████████████████████| 464/464 [00:01<00:00, 368.65it/s]


In [7]:
mvs_target_df

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679270e+09,iexn12jvq,2.0,12,178.0,100.685303,145.216174
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678937e+09,iexn01oxq,2.0,01,178.0,97.360497,141.895977
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678937e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896361
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678937e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895059
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678938e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614,F814W,1.692212e+09,iexn60olq,2.0,60,178.0,82.269096,126.780726
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692212e+09,iexn60omq,2.0,60,178.0,82.269096,126.785462
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692212e+09,iexn60onq,2.0,60,178.0,82.269081,126.783182
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692216e+09,iexn60osq,2.0,60,178.0,82.269073,126.785705


In [8]:
mvs_target_df.loc[mvs_target_df.filters=='F814W']

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678937e+09,iexn01oxq,2.0,01,178.0,97.360497,141.895977
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678937e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896361
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678937e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895059
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678938e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896192
9,J155416.68-263018.1,iexn02e3q_flc.fits,753.259138,818.650003,11.758842,156.033188,F814W,1.674509e+09,iexn02e3q,2.0,02,179.0,98.681168,143.217079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,J041548.09+291132.9,iexn59efq_flc.fits,760.651206,820.946803,5.253157,1019.242554,F814W,1.692046e+09,iexn59efq,2.0,59,178.0,82.072319,126.584263
456,J042728.00+261204.7,iexn60oiq_flc.fits,761.362280,816.845404,3.816556,1277.799561,F814W,1.692211e+09,iexn60oiq,2.0,60,178.0,82.269096,126.784238
457,J042728.00+261204.7,iexn60ojq_flc.fits,765.417200,818.406052,0.582180,1303.087524,F814W,1.692211e+09,iexn60ojq,2.0,60,178.0,82.269081,126.780863
458,J042728.00+261204.7,iexn60okq_flc.fits,763.841568,820.861524,3.087116,1275.769409,F814W,1.692211e+09,iexn60okq,2.0,60,178.0,82.269073,126.788004


In [9]:
mvs_target_df.loc[mvs_target_df.filters=='F850LP']

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679270e+09,iexn12jvq,2.0,12,178.0,100.685303,145.216174
5,J164636.12-231337.6,iexn01p1q_flc.fits,759.860639,815.166881,5.868526,225.692688,F850LP,1.678938e+09,iexn01p1q,2.0,01,178.0,97.360497,141.890608
6,J164636.12-231337.6,iexn01p2q_flc.fits,763.990396,816.515541,1.795249,207.028275,F850LP,1.678938e+09,iexn01p2q,2.0,01,178.0,97.360527,141.890860
7,J164636.12-231337.6,iexn01p3q_flc.fits,762.374175,819.081710,2.839904,220.992935,F850LP,1.678939e+09,iexn01p3q,2.0,01,178.0,97.360527,141.890628
8,J164636.12-231337.6,iexn01p4q_flc.fits,758.327324,817.628554,6.683007,212.522461,F850LP,1.678939e+09,iexn01p4q,2.0,01,178.0,97.360497,141.891153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,J041548.09+291132.9,iexn59ekq_flc.fits,760.417937,821.219986,5.600323,899.192871,F850LP,1.692051e+09,iexn59ekq,2.0,59,178.0,82.072319,126.582961
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692212e+09,iexn60omq,2.0,60,178.0,82.269096,126.785462
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692212e+09,iexn60onq,2.0,60,178.0,82.269081,126.783182
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692216e+09,iexn60osq,2.0,60,178.0,82.269073,126.785705


In [10]:
mvs_target_df.loc[~mvs_target_df.x.isna()]

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679270e+09,iexn12jvq,2.0,12,178.0,100.685303,145.216174
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678937e+09,iexn01oxq,2.0,01,178.0,97.360497,141.895977
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678937e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896361
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678937e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895059
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678938e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614,F814W,1.692212e+09,iexn60olq,2.0,60,178.0,82.269096,126.780726
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692212e+09,iexn60omq,2.0,60,178.0,82.269096,126.785462
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692212e+09,iexn60onq,2.0,60,178.0,82.269081,126.783182
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692216e+09,iexn60osq,2.0,60,178.0,82.269073,126.785705


In [11]:
elno=0
# mvs_target_df['unq_ids'] = np.nan
for TARGET in mvs_target_df.TARGET.unique():
    mvs_target_df.loc[mvs_target_df.TARGET == TARGET, 'unq_ids'] = int(elno)
    elno+=1

In [12]:
mvs_target_df = mvs_target_df.sort_values(['unq_ids','filters'])[['unq_ids', 'TARGET', 'file', 'x', 'y', 'dist', 'snr', 'filters','unittime','fitsroot','ccd','visit','exptime','pav3','rota']].reset_index(drop=True)

In [13]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F814W')]

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,0.0,J161754.11-232606.1,iexn12jpq_flc.fits,757.574858,816.657408,7.545547,87.489143,F814W,1.679268e+09,iexn12jpq,2.0,12,178.0,100.685204,145.216593
1,0.0,J161754.11-232606.1,iexn12jqq_flc.fits,761.654031,818.049830,3.346340,91.702492,F814W,1.679268e+09,iexn12jqq,2.0,12,178.0,100.685303,145.217382
2,0.0,J161754.11-232606.1,iexn12jrq_flc.fits,760.134278,820.624555,5.528430,90.594002,F814W,1.679268e+09,iexn12jrq,2.0,12,178.0,100.685303,145.217718
3,0.0,J161754.11-232606.1,iexn12jsq_flc.fits,756.183371,819.160424,8.892668,87.450500,F814W,1.679269e+09,iexn12jsq,2.0,12,178.0,100.685204,145.217124
8,1.0,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678937e+09,iexn01oxq,2.0,01,178.0,97.360497,141.895977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,56.0,J041548.09+291132.9,iexn59efq_flc.fits,760.651206,820.946803,5.253157,1019.242554,F814W,1.692046e+09,iexn59efq,2.0,59,178.0,82.072319,126.584263
456,57.0,J042728.00+261204.7,iexn60oiq_flc.fits,761.362280,816.845404,3.816556,1277.799561,F814W,1.692211e+09,iexn60oiq,2.0,60,178.0,82.269096,126.784238
457,57.0,J042728.00+261204.7,iexn60ojq_flc.fits,765.417200,818.406052,0.582180,1303.087524,F814W,1.692211e+09,iexn60ojq,2.0,60,178.0,82.269081,126.780863
458,57.0,J042728.00+261204.7,iexn60okq_flc.fits,763.841568,820.861524,3.087116,1275.769409,F814W,1.692211e+09,iexn60okq,2.0,60,178.0,82.269073,126.788004


In [14]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F850LP')]

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
4,0.0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679270e+09,iexn12jvq,2.0,12,178.0,100.685303,145.216174
5,0.0,J161754.11-232606.1,iexn12jtq_flc.fits,757.310277,817.015676,7.752466,73.934082,F850LP,1.679269e+09,iexn12jtq,2.0,12,178.0,100.685204,145.216273
6,0.0,J161754.11-232606.1,iexn12juq_flc.fits,761.477535,818.464391,3.552945,75.822121,F850LP,1.679269e+09,iexn12juq,2.0,12,178.0,100.685303,145.216521
7,0.0,J161754.11-232606.1,iexn12jwq_flc.fits,755.988511,819.732338,9.176488,67.658142,F850LP,1.679270e+09,iexn12jwq,2.0,12,178.0,100.685204,145.217031
12,1.0,J164636.12-231337.6,iexn01p1q_flc.fits,759.860639,815.166881,5.868526,225.692688,F850LP,1.678938e+09,iexn01p1q,2.0,01,178.0,97.360497,141.890608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,56.0,J041548.09+291132.9,iexn59ekq_flc.fits,760.417937,821.219986,5.600323,899.192871,F850LP,1.692051e+09,iexn59ekq,2.0,59,178.0,82.072319,126.582961
460,57.0,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692212e+09,iexn60omq,2.0,60,178.0,82.269096,126.785462
461,57.0,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692212e+09,iexn60onq,2.0,60,178.0,82.269081,126.783182
462,57.0,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692216e+09,iexn60osq,2.0,60,178.0,82.269073,126.785705


In [15]:
for target in mvs_target_df.TARGET.unique():
    if unq_target_df.loc[unq_target_df.target ==target].empty:
        index=mvs_target_df.loc[(mvs_target_df.TARGET==target)].index
        mvs_target_df.drop(index,inplace=True)
    else:
        unq_target_df.loc[unq_target_df.target ==target,'unq_ids'] = mvs_target_df.loc[mvs_target_df.TARGET == target, 'unq_ids'].values[0]
unq_target_df=unq_target_df.sort_values('unq_ids').reset_index(drop=True)
unq_target_df

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr,avg_ids
0,J161754.11-232606.1,iexn12020_drc.fits,F850LP,244.475430,-23.435078,766.494291,868.337763,20.990129,0.021352,2.242809,194.839188,0.0
1,J161754.11-232606.1,iexn12010_drc.fits,F814W,244.475431,-23.435078,766.747962,867.942406,22.213313,0.020122,2.408587,237.069580,0.0
2,J164636.12-231337.6,iexn01010_drc.fits,F814W,251.650498,-23.227155,769.100793,866.146099,20.840983,0.005901,4.008029,792.416809,1.0
3,J164636.12-231337.6,iexn01020_drc.fits,F850LP,251.650498,-23.227156,768.870891,866.488792,19.749264,0.006932,3.617601,600.396423,1.0
4,J155416.68-263018.1,iexn02010_drc.fits,F814W,238.569423,-26.505131,762.415167,869.798303,21.448087,0.010977,5.588474,411.439362,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
109,J042630.56+244355.5,iexn57020_drc.fits,F850LP,66.627360,24.732046,772.161171,860.967594,17.388929,0.000777,9.944833,5503.881348,55.0
110,J041548.09+291132.9,iexn59010_drc.fits,F814W,63.950387,29.192447,771.356066,869.879264,19.495704,0.001705,3.358237,2707.892578,56.0
111,J041548.09+291132.9,iexn59020_drc.fits,F850LP,63.950389,29.192447,771.125543,870.259044,18.268471,0.001755,3.136259,2408.281494,56.0
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496,57.0


In [16]:
mvs_target_df

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,0.0,J161754.11-232606.1,iexn12jpq_flc.fits,757.574858,816.657408,7.545547,87.489143,F814W,1.679268e+09,iexn12jpq,2.0,12,178.0,100.685204,145.216593
1,0.0,J161754.11-232606.1,iexn12jqq_flc.fits,761.654031,818.049830,3.346340,91.702492,F814W,1.679268e+09,iexn12jqq,2.0,12,178.0,100.685303,145.217382
2,0.0,J161754.11-232606.1,iexn12jrq_flc.fits,760.134278,820.624555,5.528430,90.594002,F814W,1.679268e+09,iexn12jrq,2.0,12,178.0,100.685303,145.217718
3,0.0,J161754.11-232606.1,iexn12jsq_flc.fits,756.183371,819.160424,8.892668,87.450500,F814W,1.679269e+09,iexn12jsq,2.0,12,178.0,100.685204,145.217124
4,0.0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679270e+09,iexn12jvq,2.0,12,178.0,100.685303,145.216174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,57.0,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614,F814W,1.692212e+09,iexn60olq,2.0,60,178.0,82.269096,126.780726
460,57.0,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692212e+09,iexn60omq,2.0,60,178.0,82.269096,126.785462
461,57.0,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692212e+09,iexn60onq,2.0,60,178.0,82.269081,126.783182
462,57.0,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692216e+09,iexn60osq,2.0,60,178.0,82.269073,126.785705


In [17]:
pos=0
mvs_dataframe=pd.DataFrame(columns=['unq_ids','ext', 'visit','x_f814w','y_f814w','x_f850lp','y_f850lp','fitsroot_f814w','fitsroot_f850lp', 'exptime_f814w','exptime_f850lp', 'pav3_f814w', 'pav3_f850lp', 'rota_f814w', 'rota_f850lp','flag_f814w','flag_f850lp'])
mvs_dataframe['flag_f814w']='rejected'
mvs_dataframe['flag_f850lp']='rejected'

for id in tqdm(mvs_target_df.unq_ids.unique()):
    df_F814W = mvs_target_df.loc[(mvs_target_df.unq_ids==id)&(mvs_target_df.filters=='F814W')].sort_values(['unittime'])
    df_F850LP = mvs_target_df.loc[(mvs_target_df.unq_ids==id)&(mvs_target_df.filters=='F850LP')].sort_values(['unittime'])
    num =  df_F814W.unq_ids.count() if df_F814W.unq_ids.count() >= df_F850LP.unq_ids.count() else df_F850LP.unq_ids.count()  
    for elno in range(num):
        mvs_dataframe.loc[pos,'unq_ids'] = id
        if not df_F814W.empty and elno <= df_F814W.unq_ids.count()-1:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F814W.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F814W.visit.unique()
            mvs_dataframe.loc[pos,['x_f814w','y_f814w','fitsroot_f814w','exptime_f814w','pav3_f814w','rota_f814w']] =df_F814W.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        else:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F850LP.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F850LP.visit.unique()

        if not df_F850LP.empty and elno <= df_F850LP.unq_ids.count()-1:
            mvs_dataframe.loc[pos,['x_f850lp','y_f850lp','fitsroot_f850lp','exptime_f850lp','pav3_f850lp','rota_f850lp']] =df_F850LP.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        pos+=1

mvs_dataframe

100%|██████████████████████████████████████████| 57/57 [00:00<00:00, 175.42it/s]


,avg_ids,ext,visit,x_f814w,y_f814w,x_f850lp,y_f850lp,fitsroot_f814w,fitsroot_f850lp,exptime_f814w,exptime_f850lp,pav3_f814w,pav3_f850lp,rota_f814w,rota_f850lp,flag_f814w,flag_f850lp
0,0.0,1,12,757.574858,816.657408,757.310277,817.015676,iexn12jpq,iexn12jtq,178.0,178.0,100.685204,100.685204,145.216593,145.216273,NaN,NaN
1,0.0,1,12,761.654031,818.04983,761.477535,818.464391,iexn12jqq,iexn12juq,178.0,178.0,100.685303,100.685303,145.217382,145.216521,NaN,NaN
2,0.0,1,12,760.134278,820.624555,759.949707,821.036717,iexn12jrq,iexn12jvq,178.0,178.0,100.685303,100.685303,145.217718,145.216174,NaN,NaN
3,0.0,1,12,756.183371,819.160424,755.988511,819.732338,iexn12jsq,iexn12jwq,178.0,178.0,100.685204,100.685204,145.217124,145.217031,NaN,NaN
4,1.0,1,01,760.16675,814.76482,759.860639,815.166881,iexn01oxq,iexn01p1q,178.0,178.0,97.360497,97.360497,141.895977,141.890608,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,56.0,1,59,760.651206,820.946803,760.417937,821.219986,iexn59efq,iexn59ekq,178.0,178.0,82.072319,82.072319,126.584263,126.582961,NaN,NaN
224,57.0,1,60,761.36228,816.845404,761.124038,817.287748,iexn60oiq,iexn60omq,178.0,178.0,82.269096,82.269096,126.784238,126.785462,NaN,NaN
225,57.0,1,60,765.4172,818.406052,765.226449,818.74516,iexn60ojq,iexn60onq,178.0,178.0,82.269081,82.269081,126.780863,126.783182,NaN,NaN
226,57.0,1,60,763.841568,820.861524,763.572052,821.280073,iexn60okq,iexn60osq,178.0,178.0,82.269073,82.269073,126.788004,126.785705,NaN,NaN


In [18]:
mvs_dataframe = mvs_dataframe.reset_index().rename(columns={'index':'mvs_ids'})
mvs_dataframe['unq_ids']=mvs_dataframe.unq_ids.astype(int)
mvs_dataframe


,mvs_ids,avg_ids,ext,visit,x_f814w,y_f814w,x_f850lp,y_f850lp,fitsroot_f814w,fitsroot_f850lp,exptime_f814w,exptime_f850lp,pav3_f814w,pav3_f850lp,rota_f814w,rota_f850lp,flag_f814w,flag_f850lp
0,0,0,1,12,757.574858,816.657408,757.310277,817.015676,iexn12jpq,iexn12jtq,178.0,178.0,100.685204,100.685204,145.216593,145.216273,NaN,NaN
1,1,0,1,12,761.654031,818.04983,761.477535,818.464391,iexn12jqq,iexn12juq,178.0,178.0,100.685303,100.685303,145.217382,145.216521,NaN,NaN
2,2,0,1,12,760.134278,820.624555,759.949707,821.036717,iexn12jrq,iexn12jvq,178.0,178.0,100.685303,100.685303,145.217718,145.216174,NaN,NaN
3,3,0,1,12,756.183371,819.160424,755.988511,819.732338,iexn12jsq,iexn12jwq,178.0,178.0,100.685204,100.685204,145.217124,145.217031,NaN,NaN
4,4,1,1,01,760.16675,814.76482,759.860639,815.166881,iexn01oxq,iexn01p1q,178.0,178.0,97.360497,97.360497,141.895977,141.890608,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,223,56,1,59,760.651206,820.946803,760.417937,821.219986,iexn59efq,iexn59ekq,178.0,178.0,82.072319,82.072319,126.584263,126.582961,NaN,NaN
224,224,57,1,60,761.36228,816.845404,761.124038,817.287748,iexn60oiq,iexn60omq,178.0,178.0,82.269096,82.269096,126.784238,126.785462,NaN,NaN
225,225,57,1,60,765.4172,818.406052,765.226449,818.74516,iexn60ojq,iexn60onq,178.0,178.0,82.269081,82.269081,126.780863,126.783182,NaN,NaN
226,226,57,1,60,763.841568,820.861524,763.572052,821.280073,iexn60okq,iexn60osq,178.0,178.0,82.269073,82.269073,126.788004,126.785705,NaN,NaN


In [19]:
from utils_tile import small_tiles
path2tiles='/Users/gstrampelli/PycharmProjects/Giovanni/work/out/FFP/'
bad_F814W=[124,166,192,193,194,197]
good_F814W=[4,5,6,7,14,30,43,48,50,88,90,98,116,120,122,124,126,129,141,146,148,164,165,167,168,172,174,177,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,208,209,210,211,212,216]

bad_F850LP=[166,184,185,189,190,192,197]
good_F850LP=[4,5,6,7,9,23,26,27,36,42,59,68,92,97,107,118,120,127,135,140,156,164,165,167,170,172,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,208,209,210,211,212,214,226]
dict={'bad_f814w':bad_F814W,'bad_f850lp':bad_F850LP,'good_f814w':good_F814W,'good_f850lp':good_F850LP}


mvs_dataframe = small_tiles(mvs_dataframe,path2fits, path2tiles, ['f814w','f850lp'], dict=dict,nrows=21, ncols=10)
mvs_dataframe

,mvs_ids,avg_ids,ext,visit,x_f814w,y_f814w,x_f850lp,y_f850lp,fitsroot_f814w,fitsroot_f850lp,exptime_f814w,exptime_f850lp,pav3_f814w,pav3_f850lp,rota_f814w,rota_f850lp,flag_f814w,flag_f850lp
0,0,0,1,12,757.574858,816.657408,757.310277,817.015676,iexn12jpq,iexn12jtq,178.0,178.0,100.685204,100.685204,145.216593,145.216273,good_psf,good_psf
1,1,0,1,12,761.654031,818.04983,761.477535,818.464391,iexn12jqq,iexn12juq,178.0,178.0,100.685303,100.685303,145.217382,145.216521,good_psf,good_psf
2,2,0,1,12,760.134278,820.624555,759.949707,821.036717,iexn12jrq,iexn12jvq,178.0,178.0,100.685303,100.685303,145.217718,145.216174,good_psf,good_psf
3,3,0,1,12,756.183371,819.160424,755.988511,819.732338,iexn12jsq,iexn12jwq,178.0,178.0,100.685204,100.685204,145.217124,145.217031,good_psf,good_psf
4,4,1,1,01,760.16675,814.76482,759.860639,815.166881,iexn01oxq,iexn01p1q,178.0,178.0,97.360497,97.360497,141.895977,141.890608,good_target,good_target
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,223,56,1,59,760.651206,820.946803,760.417937,821.219986,iexn59efq,iexn59ekq,178.0,178.0,82.072319,82.072319,126.584263,126.582961,good_psf,good_psf
224,224,57,1,60,761.36228,816.845404,761.124038,817.287748,iexn60oiq,iexn60omq,178.0,178.0,82.269096,82.269096,126.784238,126.785462,good_psf,good_psf
225,225,57,1,60,765.4172,818.406052,765.226449,818.74516,iexn60ojq,iexn60onq,178.0,178.0,82.269081,82.269081,126.780863,126.783182,good_psf,good_psf
226,226,57,1,60,763.841568,820.861524,763.572052,821.280073,iexn60okq,iexn60osq,178.0,178.0,82.269073,82.269073,126.788004,126.785705,good_psf,good_target


In [20]:
mvs_dataframe.loc[mvs_dataframe['flag_f814w'].isna(),'flag_f814w'] = 'rejected'
mvs_dataframe.loc[mvs_dataframe['flag_f850lp'].isna(),'flag_f850lp'] = 'rejected'


In [21]:
mvs_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_mvs_target_photometry.csv')

In [22]:
pos=0
# unq_target_df = unq_target_df[['unq_ids','ra','dec','mag_psf','e_mag_psf']].sort_values(by='unq_ids').reset_index(drop=True)
unq_dataframe=pd.DataFrame(columns=['unq_ids','ra', 'dec','m_f814w','e_f814w','m_f850lp','e_f850lp','type','target'])

for id in tqdm(unq_target_df.unq_ids.unique()):
    unq_target_df.loc[(unq_target_df.unq_ids==id),'target']=unq_target_df.loc[unq_target_df.unq_ids==id].target.values[0]
    df_F814W = unq_target_df.loc[(unq_target_df.unq_ids==id)&(unq_target_df['filter']=='F814W')]
    df_F850LP = unq_target_df.loc[(unq_target_df.unq_ids==id)&(unq_target_df['filter']=='F850LP')]
    num =  df_F814W.unq_ids.count() if df_F814W.unq_ids.count() >= df_F850LP.unq_ids.count() else df_F850LP.unq_ids.count()  
    for elno in range(num):
        unq_dataframe.loc[pos,'unq_ids'] = id
        if not df_F814W.empty:
            unq_dataframe.loc[pos,'ra'] = df_F814W.ra.unique() 
            unq_dataframe.loc[pos,'dec'] = df_F814W.dec.unique()
        elif not df_F850LP.empty:
            unq_dataframe.loc[pos,'ra'] = df_F850LP.ra.unique() 
            unq_dataframe.loc[pos,'dec'] = df_F850LP.dec.unique()
        
        unq_dataframe.loc[pos,'type'] = 1
        if np.all(mvs_dataframe.loc[mvs_dataframe.unq_ids==id,['flag_f814w','flag_f850lp']]=='bad_target'):
                unq_dataframe.loc[unq_dataframe.unq_ids==id,'type']=0
        if not df_F814W.empty and elno <= df_F814W.unq_ids.count()-1:
            unq_dataframe.loc[pos,['m_f814w','e_f814w']] =df_F814W.iloc[elno][['mag_aper','e_mag_aper']].values

        if not df_F850LP.empty and elno <= df_F850LP.unq_ids.count()-1:
            unq_dataframe.loc[pos,['m_f850lp','e_f850lp']] =df_F850LP.iloc[elno][['mag_aper','e_mag_aper']].values
        pos+=1

unq_dataframe=unq_dataframe.sort_values('unq_ids')
unq_dataframe['unq_ids']=unq_dataframe.unq_ids.astype(int)
unq_dataframe

100%|██████████████████████████████████████████| 57/57 [00:00<00:00, 498.90it/s]


,avg_ids,ra,dec,m_f814w,e_f814w,m_f850lp,e_f850lp,type,target
0,0,244.47543098532185,-23.435078302935917,22.213313,0.020122,20.990129,0.021352,1,NaN
1,1,251.6504980156288,-23.22715502396358,20.840983,0.005901,19.749264,0.006932,1,NaN
2,2,238.5694232156599,-26.505131228149015,21.448087,0.010977,20.330281,0.012019,1,NaN
3,3,241.6860922957023,-20.561958458050395,21.493397,0.011936,20.318072,0.01207,1,NaN
4,4,243.64480424478836,-24.326007804504044,21.553108,0.011095,20.299904,0.011329,1,NaN
5,5,247.04290079840197,-26.673436687805328,21.419805,0.010367,20.159706,0.01019,1,NaN
6,6,241.88167608033177,-21.781888992957537,22.182586,0.0221,20.929665,0.021179,1,NaN
7,7,238.9322686924369,-23.341438129414332,21.854202,0.015996,20.71008,0.016869,1,NaN
8,8,237.3138087651073,-24.694251812554015,22.007757,0.017982,20.915487,0.020411,1,NaN
9,9,242.3277516372519,-22.49005155708848,22.001171,0.018724,20.88359,0.020491,1,NaN


In [23]:
unq_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_unq_target_photometry.csv')

In [24]:
unq_target_df[['unq_ids','target']].to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_avg2target_ids.csv')

In [25]:
unq_target_df

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr,avg_ids
0,J161754.11-232606.1,iexn12020_drc.fits,F850LP,244.475430,-23.435078,766.494291,868.337763,20.990129,0.021352,2.242809,194.839188,0.0
1,J161754.11-232606.1,iexn12010_drc.fits,F814W,244.475431,-23.435078,766.747962,867.942406,22.213313,0.020122,2.408587,237.069580,0.0
2,J164636.12-231337.6,iexn01010_drc.fits,F814W,251.650498,-23.227155,769.100793,866.146099,20.840983,0.005901,4.008029,792.416809,1.0
3,J164636.12-231337.6,iexn01020_drc.fits,F850LP,251.650498,-23.227156,768.870891,866.488792,19.749264,0.006932,3.617601,600.396423,1.0
4,J155416.68-263018.1,iexn02010_drc.fits,F814W,238.569423,-26.505131,762.415167,869.798303,21.448087,0.010977,5.588474,411.439362,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
109,J042630.56+244355.5,iexn57020_drc.fits,F850LP,66.627360,24.732046,772.161171,860.967594,17.388929,0.000777,9.944833,5503.881348,55.0
110,J041548.09+291132.9,iexn59010_drc.fits,F814W,63.950387,29.192447,771.356066,869.879264,19.495704,0.001705,3.358237,2707.892578,56.0
111,J041548.09+291132.9,iexn59020_drc.fits,F850LP,63.950389,29.192447,771.125543,870.259044,18.268471,0.001755,3.136259,2408.281494,56.0
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496,57.0
